# demo sweep with wandb

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import os, wandb, time, random
from wandb.keras import WandbCallback
from resnet34 import ResNet34

In [2]:
# load data
from tensorflow.keras.datasets import cifar10
import keras

(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()
num_classes = 10
input_shape = X_train[0].shape
print(input_shape)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
Y_train = keras.utils.np_utils.to_categorical(Y_train, num_classes)
Y_test = keras.utils.np_utils.to_categorical(Y_test, num_classes)

(32, 32, 3)


In [3]:
# sweep config
# use WANDB_MODE=offline
sweep_config = {
  "name" : "initial_sweep",
  "method" : "random",
  "parameters" : {
    "epochs" : {
      "values" : [10, 20]
    },
    "learning_rate" :{
      "min": 0.0001,
      "max": 0.1
    },
    "optimizer" : {
        "values" : ["adam", "sgd", "nadam"]
    },
    "batch_size" : {
        "values" : [256]
    }, 
    "latent" : {
        "values" : [256, 512, 1024, 2048]
    }, 
    "activation" : {
        "values" : ["relu", "elu", "selu", "swish", "softmax"]
    }
  }, 
  "early_terminate" : {
    "type" : "hyperband",
    "s" : 2,
    "eta" : 3,
    "max_iter" : 27
  }
}

sweep_id = wandb.sweep(sweep=sweep_config, project="cifar-10-grid")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: u67y5b6w
Sweep URL: https://wandb.ai/h3x4g0n/cifar-10-grid/sweeps/u67y5b6w


In [4]:
# train function
def train():
    wandb.init(project="cifar-10-grid")
    config = wandb.config

    model = ResNet34(latent=config["latent"], n_classes=num_classes, activation=config["activation"])
    model.build(input_shape=(1, *X_train[0].shape))
    model.compile(optimizer=config["optimizer"], 
                loss="categorical_crossentropy", metrics=['accuracy'])

    model.fit(X_train, Y_train,
            epochs=config["epochs"], 
            batch_size=config["batch_size"],
            validation_data=(X_test, Y_test),
            callbacks=[WandbCallback()])

In [5]:
# sweep
wandb.agent(sweep_id, function=train)
wandb.finish()

wandb: Agent Starting Run: nhzxv1cf with config:
wandb: 	activation: selu
wandb: 	batch_size: 256
wandb: 	epochs: 50
wandb: 	latent: 1024
wandb: 	learning_rate: 0.07606747225097366
wandb: 	optimizer: nadam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: h3x4g0n. Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
/home/hexagon/miniconda3/envs/tf37/lib/python3.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


2022-06-17 23:25:05.188310: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-17 23:25:05.215748: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-17 23:25:05.215899: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-17 23:25:05.216372: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Epoch 1/50


2022-06-17 23:25:13.805510: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8400
2022-06-17 23:25:14.812507: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


196/196 [==============================] - ETA: 0s - loss: 3.4541 - accuracy: 0.1884

...

INFO:tensorflow:Assets written to: /home/hexagon/Code/m1-ml/models/wandb/run-20220617_235941-60356nj9/files/model-best/assets


INFO:tensorflow:Assets written to: /home/hexagon/Code/m1-ml/models/wandb/run-20220617_235941-60356nj9/files/model-best/assets
wandb: Adding directory to artifact (/home/hexagon/Code/m1-ml/models/wandb/run-20220617_235941-60356nj9/files/model-best)... Done. 0.7s


3125/3125 [==============================] - 76s 24ms/step - loss: 0.4737 - accuracy: 0.8399 - val_loss: 0.7329 - val_accuracy: 0.7547 - _timestamp: 1655536128.0000 - _runtime: 547.0000
Epoch 9/50
3125/3125 [==============================] - 64s 20ms/step - loss: 0.4069 - accuracy: 0.8634 - val_loss: 0.7557 - val_accuracy: 0.7538 - _timestamp: 1655536201.0000 - _runtime: 620.0000
Epoch 10/50
3125/3125 [==============================] - 65s 21ms/step - loss: 0.3450 - accuracy: 0.8842 - val_loss: 0.8095 - val_accuracy: 0.7509 - _timestamp: 1655536265.0000 - _runtime: 684.0000
Epoch 11/50
1823/3125 [================>.............] - ETA: 26s - loss: 0.2896 - accuracy: 0.9028

wandb: Ctrl + C detected. Stopping sweep.
Exception in thread Thread-199:
Traceback (most recent call last):
  File "/home/hexagon/miniconda3/envs/tf37/lib/python3.7/site-packages/wandb/agents/pyagent.py", line 300, in _run_job
    self._function()
  File "/tmp/ipykernel_13223/2828998793.py", line 15, in train
    callbacks=[WandbCallback()])
  File "/home/hexagon/miniconda3/envs/tf37/lib/python3.7/site-packages/wandb/integration/keras/keras.py", line 163, in new_v2
    return old_v2(*args, **kwargs)
  File "/home/hexagon/miniconda3/envs/tf37/lib/python3.7/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/home/hexagon/miniconda3/envs/tf37/lib/python3.7/site-packages/tensorflow/python/eager/execute.py", line 55, in quick_execute
    inputs, attrs, num_outputs)
Exception

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/hexagon/miniconda3/

accuracy,▁▃▄▅▆▆▇▇██
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▆▅▄▃▃▂▂▁▁
val_accuracy,▁▁▆▄▇▆▇███
val_loss,▆█▂▅▂▂▂▁▁▂
accuracy,0.88418
best_epoch,7
best_val_loss,0.73292
epoch,9
loss,0.34496
val_accuracy,0.7509


In [ ]:
wandb.finish()